In [2]:
#import stuff 
import wordle_words as wordle
from collections import Counter
import math
try: 
    from tqdm import tqdm
except ImportError as e :
    tqdm = lambda x: x


In [28]:
# new way of inputting information 

# Here to input information, put in your five letter guess, 
# followed by the response in the following way: 
#  put ('guess','response'), on the list, where each character of 'response' 
#  indicates the color that the corresponding character of the guess turned: 
#    *  if the Tile turned green,  put 'g'
#    *  if the Tile turned yellow, put 'y'
#    *  if the Tile turned gray,   put 'x' 
# so for example if you guessed 'tears', and the correct word was 'rebut', 
#       the response string would be 'ygxyx'

# You should always start with the guess 'tears' or 'soare' or 'trace', rather than re-running this program 

Guesses = [
    ('soare','xgxxx'),  
    ('culty','xxxyx'),
#     ('hurst','xxxxg'),
#     ('scrim','xxxyx'),
#     ('joint','xgggg')
]

In [29]:
known_letters = {}
yellow_letters = []
wrong_letters = ''
for guess,resp in Guesses: 
    for i in range(5): 
        if resp[i] == 'g': 
            known_letters[i] = guess[i]
        elif resp[i] == 'y': 
            yellow_letters.append((i,guess[i]))
        else: 
            wrong_letters += guess[i]
correct_lets = [let for _,let in yellow_letters] + list(known_letters.values())   
wrong_letters = ''.join(c for c in wrong_letters if c not in correct_lets)
print(known_letters)
print(yellow_letters)
print(wrong_letters)

{1: 'o'}
[(3, 't')]
sareculy


In [30]:
possible_words = wordle.sc_words

# start by narrowing based off of known letters 
for ind, val in known_letters.items():
    if val == '' : 
        continue 
    print(f"narrowing based off of '{val}' in spot {ind}")
    possible_words = [word for word in  possible_words if word[ind] == val]
    print(f"\t list size is {len(list(possible_words))}")

# then look at all yellow letters 
for ind, val in yellow_letters:
    if val == '' : 
        continue 
    print(f"narrowing based off of '{val}' in spot {ind}")
    possible_words = [word for word in  possible_words if val in word and word[ind] != val]
    print(f"\t list size is {len(list(possible_words))}")
# finally look at all letters not in the words 

print("narrowing words based off of incorrect letters")
for let in wrong_letters: 
    print(f"narrowing based off of incorrect letter '{let}'")
#     print(let not in known_letters.values())
#     print(list(known_letters.values()))
    possible_words = [word for word in  possible_words if let not in word or let in known_letters.values()]
#     print(f'list is {possible_words}')
print(f"\t list size is {len(list(possible_words))}")


print(f"all possible words are: {list(possible_words)}")

narrowing based off of 'o' in spot 1
	 list size is 279
narrowing based off of 't' in spot 3
	 list size is 53
narrowing words based off of incorrect letters
narrowing based off of incorrect letter 's'
narrowing based off of incorrect letter 'a'
narrowing based off of incorrect letter 'r'
narrowing based off of incorrect letter 'e'
narrowing based off of incorrect letter 'c'
narrowing based off of incorrect letter 'u'
narrowing based off of incorrect letter 'l'
narrowing based off of incorrect letter 'y'
	 list size is 5
all possible words are: ['joint', 'motif', 'point', 'toxin', 'vomit']


In [31]:
# hard mode based geussing
hard_mode = 'Default' # currently hard mode is not garunteed to be 'optimal'. 
                 # But when only a few options are avaliable, it often performs better.
use_count = False;

if hard_mode == 'Default': 
    hard_mode  = len(possible_words) <= 8
    print(f"using hard mode: {hard_mode}")
    
def calc_shannon_entropy(counts:Counter):
    all_counts = [count for _,count in counts.items()]
    all_words = sum(all_counts)
    probs = (count / all_words  for count in all_counts)
    return sum((-1*prob *math.log(prob) for prob in probs))

    
        

possible_guesses = possible_words  if hard_mode else wordle.words
best_metric = 0 
best_guesses = []
for guess in tqdm(possible_guesses):
    resps = []
    for ans in possible_words: 
        resp = ['g' if ans[i] == c else 'y' if c in ans else 'x' for i,c in enumerate(guess)]
        resps.append(''.join(resp))
    
    resp_counts = Counter(resps)
    num_uniques = len(resp_counts)
    entropy = calc_shannon_entropy(resp_counts)
    metric = num_uniques if use_count else entropy
#     print(f"{guess} has resp_count: {num_uniques} and entropy = {calc_shannon_entropy(resp_counts)} ")

    if metric > best_metric: 
        best_guesses = []
        best_metric = metric
        
    if metric >= best_metric: 
        entr = {
            "guess":        guess,
            "num_uniques":  num_uniques, 
            "resp_distro":  resp_counts,
            "shannon_entr": entropy
        }
        best_guesses.append(entr)
    
for guess in best_guesses:
    printafterline=False
    if guess['guess'] in possible_words and not hard_mode:
        printafterline = True 
        print("Reccomended word: ")
    print(guess)
    if printafterline : 
        print(" ")
    

100%|██████████| 5/5 [00:00<00:00, 14208.35it/s]

using hard mode: True
{'guess': 'joint', 'num_uniques': 5, 'resp_distro': Counter({'ggggg': 1, 'xgyxy': 1, 'xgggg': 1, 'xgyyy': 1, 'xgyxg': 1}), 'shannon_entr': 1.6094379124341005}
{'guess': 'point', 'num_uniques': 5, 'resp_distro': Counter({'xgggg': 1, 'xgyxy': 1, 'ggggg': 1, 'xgyyy': 1, 'xgyxg': 1}), 'shannon_entr': 1.6094379124341005}
